# Preparing and loading all data 
after the data is retrievded from yahoo, all data is stored in a S3 bucket in csv files, as part of the project I also converted the data into json format and stored it in the S3 bucket. The entire smoothness of process depends on the upload speed of your internet. In my case the upload speed is pretty slow, therefore it's possible to comment out the procedures in the main body. 




In [ ]:
#!pip install --upgrade pip

#!pip install pandas
#!pip install numpy
##!pip install pandas-datareader
#!pip install matplotlib
#!pip install beautifullsoup4
#!pip install scikit-learn
#!pip install sklearn

#!pip install https://github.com/matplotlib/mpl_finance/archive/master.zip
#!pip install git+https://github.com/pydata/pandas-datareader.git
    

# Import the packages once installed. 

In [11]:
import bs4 as bs
import pickle
import requests
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests
import numpy as np
import matplotlib.pyplot as plt
import shutil
import csv
import json
import glob
import boto3
import configparser
import psycopg2


# Read the configurations and get the KEY and SECRET for AWS
all configuration parameters can be stored in the dwh.cfg file and is based on a KEY = VALUE notation

In [12]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['KEY']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['SECRET']

# procedures and functions to collect and store data

## A function that writes a file into the S3 bucket

In [13]:
"""
    write_to_s3
    ___________________________________________________
    description: it writes the files in an s3 provided bucket. 
    parameters: 
        filename: the full filename, including the subfolder where the file is located in your notebook
        bucket: the name of the s3 bucket
        key: the full subfolder and filname where you want the file to be copied
    return: n/a
"""
def write_to_s3(filename, bucket, key):    
    with open(filename, 'rb') as f:
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

## The following procedure gets a list with all S&P companies
The list is retrieved from wikipedia by using beautifulsoup4 where the data is stored in a serialized file and addionally copied into the S3 bucket.

In [14]:
"""
    pickle_sp500_companies
    ___________________________________________________
    
    description: scrape the S&P500 companies table from wikipedia. Use pickle to serialize and save the table
    parameters: n/a
    return: A pickled representation of all S&P 500 companies
"""
def pickle_sp500_tickers():
    sp500 = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(sp500.text, "lxml")
    sp500_table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    
    for row in sp500_table.findAll('tr')[1:]:
        symbol       = row.findAll('td')[0].text
        company_name = row.findAll('td')[1].text
        wiki         = 'https://en.wikipedia.org' + row.findAll('td')[1].a['href']
        sector       = row.findAll('td')[3].text
        subSector    = row.findAll('td')[4].text
        
        tickers.append([symbol.replace('\n', '').replace('.','-'),company_name.replace('\n', ''),wiki.replace('\n', ''),sector.replace('\n', ''),subSector.replace('\n', '')])
    
    # write the tickers into a CSV file and upload to S3 for further processing
    df = pd.DataFrame(tickers)
    df.to_csv('stock_dfs/sp500_companies.csv',header=False,sep=";")
    write_to_s3('stock_dfs/sp500_companies.csv','cda-dend-capstone','stock_dfs/sp500_companies.csv')
    
    
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers, f)
        
    
    return tickers

#pickle_sp500_tickers() 

## The S&P companies file is used to retrieve ticker info for each company
The ticker data is stored in CSV and JSON files and uploaded to the S3 bucket.

In [15]:
"""
    get_stock_data_from_yahoo_financials
    ___________________________________________________
    
    description: This method will get a ticker for each company in the sp500tickers.picle file. 
                 For each ticker a CSV and JSON file is created. 
    parameters: reload_sp500 (True or false)
        True: it will execute the function pickle_sp500_tickers once more
        False: It's not required to retrieve each time all S&P 500 companies for this project
    return: n/a
"""
def get_stock_data_from_yahoo_financials(reload_sp500):
    if reload_sp500:
        tickers = pickle_sp500_tickers()
    else:
        with open ("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    
    # I prefer to recreate all content, at least for now. 
    if os.path.exists('stock_dfs/csv'):
        shutil.rmtree('stock_dfs/csv')
    os.makedirs('stock_dfs/csv')
    
    if os.path.exists('stock_dfs/json'):
        shutil.rmtree('stock_dfs/json')
    os.makedirs('stock_dfs/json')

    # 2000 is a nice number to start with and collects lots of information
    start = dt.datetime(2000,1,1)
    end = dt.datetime.now()
    
    for ticker in tickers:
        # build a dataframe with all ticker information from yahoo and set the date value as the index
        df = web.DataReader(ticker[0], 'yahoo', start, end)
        df.reset_index(inplace=True)
        df.set_index('Date', inplace=True)
        df.insert(0, 'company', ticker[0])
            
        # store each ticker into it's own csv file on a folder located together with the notebooks 
        df.to_csv('stock_dfs/csv/{}.csv'.format(ticker[0]))
        
        # convert the csv into json
        with open('stock_dfs/csv/{}.csv'.format(ticker[0])) as f:
            reader = csv.DictReader(f)
            rows = list(reader)  
        with open('stock_dfs/json/{}.json'.format(ticker[0]), 'w') as f:
            json.dump(rows, f)
        
        # convert the csv into json
        write_to_s3('stock_dfs/csv/{}.csv'.format(ticker[0]),'cda-dend-capstone','stock_dfs/csv/{}.csv'.format(ticker[0]))
        write_to_s3('stock_dfs/json/{}.json'.format(ticker[0]),'cda-dend-capstone','stock_dfs/json/{}.json'.format(ticker[0]))



# BEFORE YOU CAN CONTINUE 
## run the etl.py with the parameter equal to 'dim'
from a terminal run: `python etl.py dim`




# The data is loaded in the dimensional tables, let's continue with the fact tables
The fact tables are actually strategy indicators. I only implemented 3 strategies but it's possible tio implement many more

In [16]:
"""
    load_sp500_sma30_data
    ___________________________________________________
    
    description: A detailed description of the SMA 30 can be found in the 'test and plot dataset' notebook
                 This method creates a CSV for each company in CSV format the S3 bucket
    parameters: reload_sp500 (True or false)
        True: it will execute the function pickle_sp500_tickers once more
        False: It's not required to retrieve each time all S&P 500 companies for this project
    return: n/a
"""
def load_sp500_sma30_data(reload_sp500):
    config = configparser.ConfigParser()
    config.read('dwh.cfg')
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    if reload_sp500:
        tickers = pickle_sp500_tickers()
    else:
        with open ("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    
    for ticker in tickers:    
        df = pd.read_sql_query("select date, adj_close from public.sp500_tickers where company = '{}' order by date asc".format(ticker[0]), conn, index_col='date')
        df['30 SMA'] = pd.Series.rolling(df, 30).mean()
        df.dropna(inplace=True)
        #print(df.head())
        df['company'] = ticker[0]
        #print(df)
        df.to_csv('stock_dfs/sma30/{}.csv'.format(ticker[0]))

        write_to_s3('stock_dfs/sma30/{}.csv'.format(ticker[0]),'cda-dend-capstone','stock_dfs/sma30/{}.csv'.format(ticker[0]))

    conn.commit()
    conn.close()
    


    

In [17]:
"""
    load_sp500_bollinger_data
    ___________________________________________________
    
    description: A detailed description of Bollinger can be found in the 'test and plot dataset' notebook
                 This method creates a CSV for each company in CSV format the S3 bucket
    parameters: reload_sp500 (True or false)
        True: it will execute the function pickle_sp500_tickers once more
        False: It's not required to retrieve each time all S&P 500 companies for this project
    return: n/a
"""
def load_sp500_bollingerBand_data(reload_sp500):
    config = configparser.ConfigParser()
    config.read('dwh.cfg')
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    if reload_sp500:
        tickers = pickle_sp500_tickers()
    else:
        with open ("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    
    for ticker in tickers:    
        df = pd.read_sql_query("select date, adj_close from public.sp500_tickers where company = '{}' order by date asc".format(ticker[0]), conn, index_col='date')
        
        df['20_ma'] = pd.Series.rolling(df['adj_close'], 20).mean()
        
        df['20_sd'] = pd.Series.rolling(df['adj_close'], 20).std()
        
        df['Upper_Band'] = df['20_ma'] + (df['20_sd']*2)
        df['Lower_Band'] = df['20_ma'] - (df['20_sd']*2)
        df.dropna(inplace=True)
        #print(df.head())
        df['company'] = ticker[0]
        #print(df)
        df.to_csv('stock_dfs/bollinger/{}.csv'.format(ticker[0]))

        write_to_s3('stock_dfs/bollinger/{}.csv'.format(ticker[0]),'cda-dend-capstone','stock_dfs/bollinger/{}.csv'.format(ticker[0]))

    conn.commit()
    conn.close()
    




In [18]:
"""
    load_sp500_MACD_data
    ___________________________________________________
    
    description: A detailed description of the MACD can be found in the 'test and plot dataset' notebook
                 This method creates a CSV for each company in CSV format the S3 bucket
    parameters: reload_sp500 (True or false)
        True: it will execute the function pickle_sp500_tickers once more
        False: It's not required to retrieve each time all S&P 500 companies for this project
    return: n/a
"""
def load_sp500_MACD_data(reload_sp500):
    config = configparser.ConfigParser()
    config.read('dwh.cfg')
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    if reload_sp500:
        tickers = pickle_sp500_tickers()
    else:
        with open ("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
            
    # first check if the macd folder exist, delete the content if it does (at least for now)        
    if os.path.exists('stock_dfs/macd'):
        shutil.rmtree('stock_dfs/macd')
    os.makedirs('stock_dfs/macd')
        
    for ticker in tickers:    
        df = pd.read_sql_query("select date, adj_close from public.sp500_tickers where company = '{}' order by date asc".format(ticker[0]), conn, index_col='date')
        
        df['30_mavg'] = pd.Series.rolling(df['adj_close'], 30).mean()
        df['26_ema'] = pd.Series.ewm(df['adj_close'], 26).mean()
        df['12_ema'] = pd.Series.ewm(df['adj_close'], 12).mean()
        df['MACD'] = (df['12_ema'] - df['26_ema'])
        df['Signal'] = pd.Series.ewm(df['MACD'], 9).mean()
        df['Crossover'] = df['MACD'] - df['Signal']
        
        df.dropna(inplace=True)
        #print(df.head())
        df['company'] = ticker[0]
        #print(df)
        
        df.to_csv('stock_dfs/macd/{}.csv'.format(ticker[0]))

        write_to_s3('stock_dfs/macd/{}.csv'.format(ticker[0]),'cda-dend-capstone','stock_dfs/macd/{}.csv'.format(ticker[0]))

    conn.commit()
    conn.close()
    


In [19]:
def main():
    """
        main procedure
        
        Parameter:
            - nil

        Return:
            - nil
    """
    # parameter: reload_sp500
    #      True: create a sp500_companies.csv file, 
    #      False: use the existing sp500_companies.csv file
    #get_stock_data_from_yahoo_financials(False)
    #load_sp500_sma30_data(False)
    #load_sp500_bollingerBand_data(False)
    #load_sp500_macd_data(False)
    
    

if __name__ == "__main__":
    main()

# BEFORE YOU TEST AND PLOT THE DATA
## run the etl.py with the parameter equal to 'fact' to load the fact tables in the database

from a terminal run: `python etl.py fact`
